In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("train.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(["PassengerId","Name","Ticket"],axis = 1, inplace = True)

In [4]:
pd.DataFrame({"uniques":df.nunique(),
              "null count":df.isna().sum()
             })

,uniques,null count
Survived,2,0
Pclass,3,0
Sex,2,0
Age,88,177
SibSp,7,0
Parch,7,0
Fare,248,0
Cabin,147,687
Embarked,3,2


In [5]:
if "Cabin" in df.columns:
    df.drop("Cabin", axis =1, inplace = True)
    print("Dropped")


Dropped


In [6]:
df.loc[df['Embarked'].isna(), 'Embarked']  = df['Embarked'].mode()[0]

In [7]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

In [8]:
# df[df.Age.isna()]
df.loc[df['Age'].isna(), 'Age'] = df['Age'].mode()[0]

In [9]:
pd.DataFrame({"uniques":df.nunique(),
              "null count":df.isna().sum()
             })

,uniques,null count
Survived,2,0
Pclass,3,0
Sex,2,0
Age,88,0
SibSp,7,0
Parch,7,0
Fare,248,0
Embarked,3,0


In [10]:
from sklearn.preprocessing import LabelEncoder
labelEncoder_gender =  LabelEncoder()
df["Sex"] = labelEncoder_gender.fit_transform(df["Sex"])
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

In [11]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [12]:
target = "Survived"
X = df.drop(target,axis=1)
y =df[target]

In [13]:
X = df[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,2
887,1,0,19.0,0,0,30.0000,2
888,3,0,24.0,1,2,23.4500,2
889,1,1,26.0,0,0,30.0000,0


In [14]:

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
# # Feature scaling
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [15]:
# Compare classification algorithms
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [16]:
classification_models = []
classification_models.append(('Logistic Regression', LogisticRegression(solver="liblinear")))
classification_models.append(('K Nearest Neighbor', KNeighborsClassifier(n_neighbors=5, metric="minkowski",p=2)))
classification_models.append(('Kernel SVM', SVC(kernel = 'rbf',gamma='scale')))
classification_models.append(('Naive Bayes', GaussianNB()))
classification_models.append(('Decision Tree', DecisionTreeClassifier(criterion = "entropy")))


# Accuracy

In [17]:
for name, model in classification_models:
  kfold = KFold(n_splits=10, random_state=(7), shuffle=(True))
  result = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
  # print('Results : ', result)
  print("%s: Mean Accuracy = %.2f%% - SD Accuracy = %.2f%%" % (name, result.mean()*100, result.std()*100))

Logistic Regression: Mean Accuracy = 79.12% - SD Accuracy = 4.24%
K Nearest Neighbor: Mean Accuracy = 70.92% - SD Accuracy = 6.55%
Kernel SVM: Mean Accuracy = 67.78% - SD Accuracy = 6.03%
Naive Bayes: Mean Accuracy = 78.56% - SD Accuracy = 4.04%
Decision Tree: Mean Accuracy = 78.12% - SD Accuracy = 3.76%


# Precision

In [18]:
for name, model in classification_models:
  kfold = KFold(n_splits=10, random_state=(7), shuffle=(True))
  result = cross_val_score(model, X, y, cv=kfold, scoring='precision')
  # print('Results : ', result)
  print("%s: Mean Precision = %.2f%% - SD Precision = %.2f%%" % (name, result.mean()*100, result.std()*100))

Logistic Regression: Mean Precision = 74.83% - SD Precision = 6.09%
K Nearest Neighbor: Mean Precision = 63.05% - SD Precision = 7.82%
Kernel SVM: Mean Precision = 70.19% - SD Precision = 12.10%
Naive Bayes: Mean Precision = 71.87% - SD Precision = 7.24%
Decision Tree: Mean Precision = 71.32% - SD Precision = 6.91%


# Recall

In [19]:
for name, model in classification_models:
  kfold = KFold(n_splits=10, random_state=(7), shuffle=(True))
  result = cross_val_score(model, X, y, cv=kfold, scoring='recall')
  # print('Results : ', result)
  print("%s: Mean Recall = %.2f%% - SD Recall = %.2f%%" % (name, result.mean()*100, result.std()*100))

Logistic Regression: Mean Recall = 68.93% - SD Recall = 5.67%
K Nearest Neighbor: Mean Recall = 59.28% - SD Recall = 9.50%
Kernel SVM: Mean Recall = 29.88% - SD Recall = 6.87%
Naive Bayes: Mean Recall = 73.28% - SD Recall = 4.11%
Decision Tree: Mean Recall = 72.42% - SD Recall = 3.52%
